This notebook allows you to compare the performance of segmentations generated from new affinity networks against the old one trained on JWR dataset. It assumes that the error analysis modules in the ecerebellum package have been run and results have been saved in the respective folders.

In [1]:
import numpy as np
import json

In [2]:
eval_chunks = range(12,16) # IDs of validation chunks
train_chunks = range(0,12) # IDs of training chunks
block_size = 60 # number of slices per chunk
resdir = '/mnt/coxfs01/donglai/srujan/err-analysis/' # base directory with error analysis results

In [3]:
def get_erl_stats(resdir, seg_names, block_size, aff_offset):
    """
    Reads err-analysis results and evaluates mean and std ERL across many chunks
    """
    vi = []
    erl_gt = []
    erl_pred = []
    for i in range(len(seg_names)):
        print "On evaluated chunk: %d"%i
        base_seg = seg_names[i]
        vi_dict = json.load(open(resdir+base_seg+"/vi.json"))
        vi.append((vi_dict["VI split"], vi_dict["VI merge"]))
        print "VI: ", vi[i]
        erl_dict = json.load(open(resdir+base_seg+"/skeleton-analysis-summary.json"))
        erl_gt.append(erl_dict["results"]["erl-gt"])
        print "GT ERL in nm: %d"%(erl_gt[i])
        erl_pred.append(erl_dict["results"]["erl-pred"])
        print "Predicted ERL in nm: %d"%erl_pred[i]
    erl_gt_mean = np.mean(erl_gt)
    std_gt_mean = np.std(erl_gt)
    erl_pred_mean = np.mean(erl_pred)
    std_pred_mean = np.std(erl_pred)
    print "Mean GT ERL in nm: %d +/- %d"%(erl_gt_mean, std_gt_mean)
    print "Mean pred ERL in nm: %d +/- %d"%(erl_pred_mean, std_pred_mean)
    return erl_pred_mean, std_pred_mean#, np.sum(erl_gt), np.sum(erl_pred)

## JWR model on current test chunks

In [4]:
wz_thresh = 0.5
aff_offset = 14
zz = [chunk_id*block_size + aff_offset for chunk_id in eval_chunks]
seg_names = ["waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz[i]) for i in range(len(eval_chunks))]
erl_jwr_test, std_jwr_test = get_erl_stats(resdir,seg_names, block_size, aff_offset)

On evaluated chunk: 0
VI:  (0.6249742436145521, 0.2612395624648516)
GT ERL in nm: 3303
Predicted ERL in nm: 1543
On evaluated chunk: 1
VI:  (0.48167293075122775, 0.2046038754984993)
GT ERL in nm: 2768
Predicted ERL in nm: 1078
On evaluated chunk: 2
VI:  (0.6155921005704638, 0.3002865526185817)
GT ERL in nm: 2789
Predicted ERL in nm: 871
On evaluated chunk: 3
VI:  (0.2886886095581719, 0.062055236428578985)
GT ERL in nm: 2376
Predicted ERL in nm: 1479
Mean GT ERL in nm: 2809 +/- 329
Mean pred ERL in nm: 1243 +/- 279


## JWR model on current training chunks

In [5]:
wz_thresh = 0.5
aff_offset = 14
zz = [chunk_id*block_size + aff_offset for chunk_id in train_chunks]
seg_names = ["waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz[i]) for i in range(len(train_chunks))]
erl_jwr_train, std_jwr_train = get_erl_stats(resdir,seg_names, block_size, aff_offset)

On evaluated chunk: 0
VI:  (0.08441740089257588, 0.007626745858172034)
GT ERL in nm: 2005
Predicted ERL in nm: 1661
On evaluated chunk: 1
VI:  (0.11385432584697436, 0.009933146890894319)
GT ERL in nm: 2299
Predicted ERL in nm: 1693
On evaluated chunk: 2
VI:  (0.3095177768012949, 0.11856814009009421)
GT ERL in nm: 2877
Predicted ERL in nm: 1450
On evaluated chunk: 3
VI:  (0.35223887775989937, 0.13939526327023172)
GT ERL in nm: 3227
Predicted ERL in nm: 1406
On evaluated chunk: 4
VI:  (0.6920218043220697, 0.10305426689668407)
GT ERL in nm: 2976
Predicted ERL in nm: 778
On evaluated chunk: 5
VI:  (0.6140835097411244, 0.21371108306675823)
GT ERL in nm: 4032
Predicted ERL in nm: 1460
On evaluated chunk: 6
VI:  (0.3425577640998605, 0.06824590574256541)
GT ERL in nm: 2417
Predicted ERL in nm: 1164
On evaluated chunk: 7
VI:  (0.2627277007388039, 0.05366486375807491)
GT ERL in nm: 2078
Predicted ERL in nm: 1645
On evaluated chunk: 8
VI:  (0.3230931044722458, 0.03600218501039468)
GT ERL in nm: 1

## UnetPNI3DM2 - 11K epochs

In [6]:
seg_type = 'wtdil2'
wz_thresh = 0.5
zz = [chunk_id*block_size for chunk_id in eval_chunks]
seg_names = ["retrained-"+seg_type+"-%04d-wz%.2f"%(zz[i], wz_thresh) for i in range(len(eval_chunks))]
erl_jwr, std_jwr = get_erl_stats(resdir,seg_names, block_size, aff_offset)

On evaluated chunk: 0
VI:  (0.6334221632069712, 0.4837357038319403)
GT ERL in nm: 3303
Predicted ERL in nm: 1474
On evaluated chunk: 1
VI:  (1.1411665942507934, 0.9876153705385816)
GT ERL in nm: 2768
Predicted ERL in nm: 781
On evaluated chunk: 2
VI:  (0.6832663498113299, 0.35468840385482203)
GT ERL in nm: 2789
Predicted ERL in nm: 1004
On evaluated chunk: 3
VI:  (0.24291393107932652, 0.06120300584460292)
GT ERL in nm: 2376
Predicted ERL in nm: 1635
Mean GT ERL in nm: 2809 +/- 329
Mean pred ERL in nm: 1223 +/- 345


## Unet3DPniM2 - 49K epochs

In [7]:
seg_type = '49Ktrain'
wz_thresh = 0.5
zz = [chunk_id*block_size for chunk_id in eval_chunks]
seg_names = ["retrained-"+seg_type+"-%04d-wz%.2f"%(zz[i], wz_thresh) for i in range(len(eval_chunks))]
erl_jwr, std_jwr = get_erl_stats(resdir,seg_names, block_size, aff_offset)

On evaluated chunk: 0
VI:  (0.4562201154182672, 0.9971963062090315)
GT ERL in nm: 3303
Predicted ERL in nm: 1210
On evaluated chunk: 1
VI:  (0.4221393722892554, 0.8698675382109224)
GT ERL in nm: 2768
Predicted ERL in nm: 806
On evaluated chunk: 2
VI:  (0.5432551474648435, 0.5770045089681198)
GT ERL in nm: 2789
Predicted ERL in nm: 873
On evaluated chunk: 3
VI:  (0.1993912599505692, 0.0928270150688557)
GT ERL in nm: 2376
Predicted ERL in nm: 1605
Mean GT ERL in nm: 2809 +/- 329
Mean pred ERL in nm: 1124 +/- 317


## UnetFiber - 12K epochs

In [9]:
seg_type = '12Ktrain'
wz_thresh = 0.5
zz = [chunk_id*block_size for chunk_id in eval_chunks]
seg_names = ["retrained-"+seg_type+"-%04d-wz%.2f"%(zz[i], wz_thresh) for i in range(len(eval_chunks))]
erl_jwr, std_jwr = get_erl_stats(resdir,seg_names, block_size, aff_offset)

On evaluated chunk: 0
VI:  (0.4896863462445564, 0.8585366370428827)
GT ERL in nm: 3303
Predicted ERL in nm: 1247
On evaluated chunk: 1
VI:  (0.7305311816485477, 0.8552270099191793)
GT ERL in nm: 2768
Predicted ERL in nm: 763
On evaluated chunk: 2
VI:  (0.6323407030539118, 0.8016363419502773)
GT ERL in nm: 2789
Predicted ERL in nm: 753
On evaluated chunk: 3
VI:  (0.2374998483233842, 0.3080794344300453)
GT ERL in nm: 2376
Predicted ERL in nm: 1310
Mean GT ERL in nm: 2809 +/- 329
Mean pred ERL in nm: 1018 +/- 261
